# Udacity Project 3

# Gather

In [1]:
#imports that will be used for this file:
import pandas as pd
import requests
import os
import csv
import tweepy
import json
import re
import tweepy
import json
import time
import numpy as np
import math

In [2]:
df_arch = pd.read_csv("twitter-archive-enhanced.csv", sep= ",")
id_list = df_arch["tweet_id"]
#Testing with small batches below
#id_list = df_arch["tweet_id"][:100]
#id_list = [892420643555336193, 0]
#id_list = [892420643555336193, 892177421306343426, 0]

Image Prediction File Download

In [3]:
#download the file.
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
r = requests.get(url)


In [4]:
#save the .tsv file into the notebook folder. 
with open('file_2', 'w') as f:
    writer = csv.writer(f)
    reader = csv.reader(r.text.splitlines())
    
    for row in reader:
            writer.writerow(row)
#https://stackoverflow.com/questions/45978295/saving-a-downloaded-csv-file-using-python

In [5]:
#put it into a panda dataframe.
df_img_algo = pd.read_csv("file_2", sep= "\t")
df_img_algo.head()


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


Third file from the twitter API

In [6]:
#consumer_key = 'MYKEY'
#consumer_secret = 'MYKEY'
#access_token = 'MYKEY-MYKEY'
#access_secret = 'MYKEY'
##file and list setup
#templist = []
#f = open("tweet_json.txt", "a+")
#count = 0
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)
#api = tweepy.API(auth, wait_on_rate_limit= True, wait_on_rate_limit_notify= True)
#for id in id_list:
#    count += 1
#    try:
#        start = time.time()
#        tweet = api.get_status(id)
#        templist.append(tweet)
#        end = time.time()
#        print(count, "ID: ", id, "TIME: ", end - start)
#    except: 
#        print(count, "ID: ", id, " could not be found.")
#        continue

#tweet = api.get_status(666020888022790149)


In [7]:
#templist_json = []
#for item in templist:
#    templist_json.append(item._json)
#with open("tweet_json.txt", "w+") as f:
#    f.write(json.dumps(templist_json, indent = 4))
    
    

In [8]:
#create dataframe in Pandas. 
db_list = []

with open("tweet_json.txt") as f:
    json_temp = json.load(f)
    for item in json_temp:
        tweet_id = item["id"]
        tweet_text = item["text"]
        url = tweet_text[tweet_text.find("https"):]
        favorite_count = item["favorite_count"]
        retweet_count = item["retweet_count"] 
        hashtags = item["entities"]["hashtags"]
        followers_count = item["user"]["followers_count"]
        created_at = item["created_at"]
        
        db_list.append({"tweet_id": str(tweet_id),
                            "favorite_count": int(favorite_count),
                            "retweet_count": int(retweet_count),
                            "url": url,
                            "hashtags": hashtags,
                            "created_at": created_at,
                            "followers_count": followers_count
                            })
        
        df_tweet_json = pd.DataFrame(db_list, columns = ["tweet_id", "retweet_count", "favorite_count", "followers_count", 
                                                        "hashtags", "url", "created_at"])


In [9]:
#At this point we have three dataframes: df_arch, df_img_algo, df_tweet_json.
#We need to Assess the three data frames and clean them. 


# Assess

### Twitter .csv file

In [10]:
#First data type is the twitter archive which was lifted from the .csv file. 
df_arch.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


The df_arch dataframe has some noticible tidiness and quality issues. 
Firstly, the timestamp has a date and time. Ideally the date should be seperated out. Some quality issues I see visually are the conconsistencies between "null" data. None vs NaN can be unified into a "null" data. 

In [11]:
df_arch.info()
#This simple look up details tweet_id is incorrectly a integer. 
#It should be, like the zip code example we studied in the text, a string.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [12]:
df_arch.describe()
#This also reveals something erroneous about the data. THe denominator should be out of 10.
#But the mean is 10.45. It's also odd that the max is 2356.000000 and the min is 0.

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


# Issues with Twitter Archieve Data
### Tidiness
1)Date and Time should be seperated in "timestamp" or the datatype should be converted. 

2)Columns related to retweet should be dropped because there's not enough meaningful data.  

3)There should be a rating_value column that explains what the actual rating is in ratio. 

4) The dog names can be cobined into one. 
### Quality
1)href should be removed from the "source" column. It should just display the device. 

2)tweet_ID should be string. (like the zipcode example from the lsson)

3)Dog names have None. That should be converted into null. 

4)The reviewer pointed out that some ID's such had an incorrect rating lifted from the 'text' field. 

5)Rows that are retweets should be removed. 

# Image Data

In [13]:
#We can load the dataframe and take a look at the 
df_img_algo.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## Quality Issue with Image Data: 
1)There isn't enough Hashtag Data. This should be removed entirely.
2)Tweet_id should be strings. 
3)Inconsistent capitalizations on "p" data. 
4)The "p" data is not descriptive enough.


# Tweets API Data


In [14]:
#visual examination first
df_tweet_json.head()

,tweet_id,retweet_count,favorite_count,followers_count,hashtags,url,created_at
0,892420643555336193,8642,38967,6551974,[],https://t.co/MgUWQ76dJU,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,6351,33354,6551974,[],https://t.co/aQFSeaCu9L,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,4213,25137,6551974,[],https://t.co/r0YlrsGCgy,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,8759,42299,6551975,[],https://t.co/tD36da7qLQ,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,9521,40470,6551975,[],https://t.co/0g0KMIVXZ3,Sat Jul 29 16:00:24 +0000 2017


In [15]:
#tweet_ID should be a string, and hashtags return a "[]" instead of Null. 
#also I am seeing that created_at time should be seperated out to date and time and day of the week.
df_tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 7 columns):
tweet_id           2345 non-null object
retweet_count      2345 non-null int64
favorite_count     2345 non-null int64
followers_count    2345 non-null int64
hashtags           2345 non-null object
url                2345 non-null object
created_at         2345 non-null object
dtypes: int64(3), object(4)
memory usage: 128.3+ KB


In [16]:
#the hashtag looks like there aren't many values. I created a simple loop to search
#for any legit dictionaries, There were only 19 enteries so I will delete this column. It's not very meaningful. 
count = 0
for item in df_tweet_json["hashtags"]:
    if len(item) > 0:
        count += 1
        print (item)
print ("total", count)

[{'text': 'BATP', 'indices': [21, 26]}]
[{'text': 'PrideMonthPuppo', 'indices': [29, 45]}]
[{'text': 'dogsatpollingstations', 'indices': [21, 43]}]
[{'text': 'PrideMonth', 'indices': [66, 77]}]
[{'text': 'PrideMonth', 'indices': [39, 50]}]
[{'text': 'ScienceMarch', 'indices': [36, 49]}]
[{'text': 'K9VeteransDay', 'indices': [40, 54]}]
[{'text': 'GoodDogs', 'indices': [62, 71]}]
[{'text': 'WKCDogShow', 'indices': [26, 37]}]
[{'text': 'BellLetsTalk', 'indices': [42, 55]}]
[{'text': 'WomensMarch', 'indices': [62, 74]}]
[{'text': 'ImWithThor', 'indices': [0, 11]}]
[{'text': 'BarkWeek', 'indices': [0, 9]}]
[{'text': 'BarkWeek', 'indices': [33, 42]}]
[{'text': 'BarkWeek', 'indices': [32, 41]}]
[{'text': 'FinalFur', 'indices': [40, 49]}]
[{'text': 'LoveTwitter', 'indices': [61, 73]}]
[{'text': 'notallpuppers', 'indices': [101, 115]}]
[{'text': 'NoDaysOff', 'indices': [98, 108]}, {'text': 'swole', 'indices': [109, 115]}]
total 19


# Issues with Twitter API Data: 

### Quality 
1)There isn't enough Hashtag Data. This should be removed entirely.

2)Tweet_id should be strings. 

### Tidiness Issues with Twitter API Data: 
1) created_at should either be parsed out to multiple fields or the data type should change to date. 

# Clean : Twitter Archive Data

In [17]:
#starting with copying df_arch
df_arch_clean = df_arch.copy()

### Issue
tweet_id should be strings, not integers. 


### Code

In [18]:
df_arch_clean["tweet_id"] = df_arch_clean["tweet_id"].astype("str")

### Test

In [19]:
type(df_arch_clean["tweet_id"])

pandas.core.series.Series

### Issue
Web Source should be just have the device, web ect. 


In [20]:
df_arch_clean["source"].value_counts()
#this explains that there are only four sources:iPhone, Vine, Twitter Web, and TweetDeck. 


<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

### Code

In [21]:
df_arch_clean['source'] = df_arch_clean['source'].str.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'iPhone')
df_arch_clean['source'] = df_arch_clean['source'].str.replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>', 'Vine')
df_arch_clean['source'] = df_arch_clean['source'].str.replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'Web')
df_arch_clean['source'] = df_arch_clean['source'].str.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'TweetDeck')

In [22]:
df_arch_clean['source'] = df_arch_clean['source'].astype('category')

### Test

In [23]:
df_arch_clean['source'].value_counts()
#before the data source as category, it was int.

iPhone       2221
Vine           91
Web            33
TweetDeck      11
Name: source, dtype: int64

### Issue 

In [24]:
# retweets should be deleted. retweets can be discovered through running the below:
df_arch_clean[df_arch_clean['retweeted_status_timestamp'].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,iPhone,RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,iPhone,RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,iPhone,RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,iPhone,RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,iPhone,RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,iPhone,RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,iPhone,RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,iPhone,RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,iPhone,RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,iPhone,RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


### Code

In [25]:
del_list = []
del_list = df_arch_clean[df_arch_clean['retweeted_status_timestamp'].notnull()]['tweet_id']
#this list contains the user id of the tweets that should be deleted.
del_index = del_list.index
#I can just drop the del index!
df_arch_clean = df_arch_clean.drop(del_index)


### Test

In [26]:
df_arch_clean[df_arch_clean['retweeted_status_timestamp'].notnull()]
#Got nothing back! it worked!

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Issue 
Retweet columns, and hashtag columns should be dropped. There's not enough on these columns:
in_reply_to_user_id
in_reply_to_status_id
retweeted_status_id
retweeted_status_user_id
retweeted_status_timestamp

In [27]:
#df_arch_clean['in_reply_to_user_id'].value_counts()
#df_arch_clean['in_reply_to_status_id'].value_counts()
#df_arch_clean['retweeted_status_id'].value_counts()
#df_arch_clean['retweeted_status_user_id'].value_counts()
#df_arch_clean['retweeted_status_timestamp'].value_counts()

### Code

In [28]:
df_arch_clean = df_arch_clean.drop('in_reply_to_user_id', axis = 1)
df_arch_clean = df_arch_clean.drop('in_reply_to_status_id', axis = 1)
df_arch_clean = df_arch_clean.drop('retweeted_status_id', axis = 1)
df_arch_clean = df_arch_clean.drop('retweeted_status_user_id', axis = 1)
df_arch_clean = df_arch_clean.drop('retweeted_status_timestamp', axis = 1)

### Test

In [29]:
df_arch_clean.info()
#columns are gone!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null object
source                2175 non-null category
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: category(1), int64(2), object(9)
memory usage: 206.2+ KB


### Issue
Time stamp shows up with date and time. initlally I tried to seperate the time vs date by writing the seperate code below. I suppose you can create a seperate column for Date and Time, but there is a better way! I can just change the data format to "datetime64".


In [30]:
#parsing out date vs time in "timestamp":
date = []
time = []
for line in df_arch_clean["timestamp"]:
    n = line.split()[0]
    t = line.split()[1]
    date.append(n)
    time.append(t)
#printing out date and time will show that the two lists has the time and date separated. 

### Code

In [31]:
df_arch_clean.timestamp = pd.to_datetime(df_arch_clean.timestamp)

### Test

In [32]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null datetime64[ns]
source                2175 non-null category
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: category(1), datetime64[ns](1), int64(2), object(8)
memory usage: 206.2+ KB


### Issue

In [33]:
df_arch_clean.loc[df_arch_clean.tweet_id== '681340665377193984', 'text']
#some ratings were incorrectly extracted. For example ID: 681340665377193984 should be 9.5 but only 5 was picked up.

1689    I've been told there's a slight possibility he...
Name: text, dtype: object

### Code

In [34]:
#change the denominator and numerator as data type: floating point because these values are decimals
df_arch_clean['rating_denominator'] = df_arch_clean['rating_denominator'].astype('float')
df_arch_clean['rating_numerator'] = df_arch_clean['rating_numerator'].astype('float')


In [35]:
#view data from that has a decimal value from the "text". The reviewer pointed out that the the regular expression made an error and only extrate one number. 

df_arch_clean[df_arch_clean.text.str.contains(r"([0-9]+\.[0-9]*\/\d+)")]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
45,883482846933004288,2017-07-08 00:28:19,iPhone,This is Bella. She hopes her smile made you sm...,https://twitter.com/dog_rates/status/883482846...,5.0,10.0,Bella,None,None,None,None
695,786709082849828864,2016-10-13 23:23:56,iPhone,"This is Logan, the Chow who lived. He solemnly...",https://twitter.com/dog_rates/status/786709082...,75.0,10.0,Logan,None,None,None,None
763,778027034220126208,2016-09-20 00:24:34,iPhone,This is Sophie. She's a Jubilant Bush Pupper. ...,https://twitter.com/dog_rates/status/778027034...,27.0,10.0,Sophie,None,None,pupper,None
1689,681340665377193984,2015-12-28 05:07:27,iPhone,I've been told there's a slight possibility he...,NaN,5.0,10.0,None,None,None,None,None
1712,680494726643068929,2015-12-25 21:06:00,iPhone,Here we have uncovered an entire battalion of ...,https://twitter.com/dog_rates/status/680494726...,26.0,10.0,None,None,None,None,None


These fields ID's are problematic.

883482846933004288  :Rating 13.5/10 

832215909146226688 :Rating 9.75/10

786709082849828864 :Rating 9.75/10

778027034220126208 :Rating 11.27/10

681340665377193984 :Rating 9.5/10

680494726643068929 :Rating 11.26/10

In [36]:
#df_arch_clean[df_arch_clean.tweet_id == '883482846933004288'].rating_numerator = 13.5
#this didn't work. 
#df_arch_clean.loc[df_arch_clean.tweet_id == '883482846933004288']=11.27
#this didn't work either. 
#http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

df_arch_clean.loc[df_arch_clean.tweet_id == '883482846933004288', 'rating_numerator'] = 13.5
df_arch_clean.loc[df_arch_clean.tweet_id == '832215909146226688', 'rating_numerator'] = 9.75
df_arch_clean.loc[df_arch_clean.tweet_id == '786709082849828864', 'rating_numerator'] = 9.75
df_arch_clean.loc[df_arch_clean.tweet_id == '778027034220126208', 'rating_numerator'] = 11.27
df_arch_clean.loc[df_arch_clean.tweet_id == '681340665377193984', 'rating_numerator'] = 9.5
df_arch_clean.loc[df_arch_clean.tweet_id == '680494726643068929', 'rating_numerator'] = 11.26

### Test

In [37]:
df_arch_clean[df_arch_clean['tweet_id']== '883482846933004288'].rating_numerator
#worked

45    13.5
Name: rating_numerator, dtype: float64

### Issue
Rating as numerator/denominator is missing. 

### Code

In [38]:
df_arch_clean['rating_value'] = df_arch_clean['rating_numerator'] / df_arch_clean['rating_denominator']

### Test

In [39]:
df_arch_clean['rating_value'].head()
#It worked!

0    1.3
1    1.3
2    1.2
3    1.3
4    1.2
Name: rating_value, dtype: float64

### Issue 
The following columns have None instead of Null:
name doggo floofer pupper puppo

This column can be combined as one dog_stage

In [40]:
#df_arch_clean[df_arch_clean['doggo'] == 'None']
#df_arch_clean.doggo.head()

In [41]:
#replace 'None' with np.NaN
#https://stackoverflow.com/questions/34794067/how-to-set-a-cell-to-nan-in-a-pandas-dataframe
df_arch_clean.loc[df_arch_clean.doggo=='None', 'doggo'] = np.NaN
df_arch_clean.loc[df_arch_clean.floofer=='None', 'floofer'] = np.NaN
df_arch_clean.loc[df_arch_clean.pupper=='None', 'pupper'] = np.NaN
df_arch_clean.loc[df_arch_clean.puppo=='None', 'puppo'] = np.NaN


In [42]:
df_arch_clean[df_arch_clean['doggo'] == 'None']
#nothing shows up!

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating_value


In [43]:
df_arch_clean['dog_stage'] = df_arch_clean['text'].str.extract('(doggo|floofer|pupper|puppo)', expand=True)
#tried using melt and didn't work. Value got doubled...so I had to get creative with strings. 

In [44]:
#get rid of the columns
df_arch_clean = df_arch_clean.drop('doggo', axis=1)
df_arch_clean = df_arch_clean.drop('floofer', axis=1)
df_arch_clean = df_arch_clean.drop('pupper', axis=1)
df_arch_clean = df_arch_clean.drop('puppo', axis=1)

In [45]:
# and we need to use the 'melt' to combine these into a single column
#df_arch_clean['dog_stage'] = np.NaN
#df_arch_clean = pd.melt(df_arch_clean, 
#                        id_vars=['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
#       'rating_numerator', 'rating_denominator', 'name', 'rating_value'],
#                        var_name= 'dog_stage',
#                        value_name= 'temp')
#df_arch_clean = df_arch_clean.drop('temp', axis=1)


In [46]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null datetime64[ns]
source                2175 non-null category
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null float64
rating_denominator    2175 non-null float64
name                  2175 non-null object
rating_value          2175 non-null float64
dog_stage             364 non-null object
dtypes: category(1), datetime64[ns](1), float64(3), object(5)
memory usage: 172.2+ KB


# Clean: Image Data

In [47]:
#next we move on to the df_image_algo_clean data
#first, copy the data. 
df_img_algo_clean = df_img_algo.copy()
df_img_algo_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Issue
tweet_id should be strings

### Code 

In [48]:
df_img_algo_clean.tweet_id = df_img_algo_clean.tweet_id.astype(str)

### Test

In [49]:
df_img_algo_clean.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


### Issue
The following columns are not descriptive:
p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog, 

### Code

In [50]:
df_img_algo_clean.rename(columns={'p1' : 'prediction_1',
                                  'p2' : 'prediction_2',
                                  'p3' : 'prediction_3',
                                  'p1_conf' : 'p1_confidence',
                                  'p2_conf' : 'p2_confidence',
                                  'p3_conf' : 'p3_confidence',
                                  'p1_dog' : 'predict_1_breed',
                                  'p2_dog' : 'predict_2_breed',
                                  'p3_dog' : 'predict_3_breed',
                                 }, inplace = True)
#https://chrisalbon.com/python/data_wrangling/pandas_rename_multiple_columns/

### Test

In [51]:
df_img_algo_clean.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'prediction_1', 'p1_confidence',
       'predict_1_breed', 'prediction_2', 'p2_confidence', 'predict_2_breed',
       'prediction_3', 'p3_confidence', 'predict_3_breed'],
      dtype='object')

### change values into lower case in the following columns: 
prediction_1
prediction_2
prediction_3

In [52]:
df_img_algo_clean['prediction_1'] = df_img_algo_clean['prediction_1'].str.lower()
df_img_algo_clean['prediction_2'] = df_img_algo_clean['prediction_2'].str.lower()
df_img_algo_clean['prediction_3'] = df_img_algo_clean['prediction_3'].str.lower()

In [53]:
df_img_algo_clean.head()

,tweet_id,jpg_url,img_num,prediction_1,p1_confidence,predict_1_breed,prediction_2,p2_confidence,predict_2_breed,prediction_3,p3_confidence,predict_3_breed
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,welsh_springer_spaniel,0.465074,True,collie,0.156665,True,shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,german_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,rottweiler,0.243682,True,doberman,0.154629,True


# Clean: Twitter JSON Data

In [54]:
#lastly we move on to df_tweet_json. 
df_tweet_json_clean = df_tweet_json.copy()

### Issue
There isn't enough data on hashtag column. We need to remove them.

### Code

In [55]:
df_tweet_json_clean = df_tweet_json_clean.drop('hashtags', axis = 1)

### Test

In [56]:
df_tweet_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 6 columns):
tweet_id           2345 non-null object
retweet_count      2345 non-null int64
favorite_count     2345 non-null int64
followers_count    2345 non-null int64
url                2345 non-null object
created_at         2345 non-null object
dtypes: int64(3), object(3)
memory usage: 110.0+ KB


### Issue
the created_at column has the wrong type. 

### Code

In [57]:
#df_tweet_json_clean['created_at'].astype(datetime)
df_tweet_json_clean.created_at = pd.to_datetime(df_tweet_json_clean.created_at)

### Test

In [58]:
df_tweet_json_clean['created_at'].head()
#looks great

0   2017-08-01 16:23:56
1   2017-08-01 00:17:27
2   2017-07-31 00:18:03
3   2017-07-30 15:58:51
4   2017-07-29 16:00:24
Name: created_at, dtype: datetime64[ns]

### Saving the files to CSV

In [59]:
df_arch_clean.to_csv('twitter_archive_master.csv')
df_img_algo_clean.to_csv('df_img_algo_clean')
df_tweet_json_clean.to_csv('df_tweet_json_clean')


In [60]:
#df_arch_clean.head(50)
#df_img_algo_clean.head(50)
#df_tweet_json_clean.head(50)

# Analysis 1)

When do people tweet? 

In [61]:
df1 = df_arch_clean.copy()
df1['timestamp'] = pd.to_datetime(df1['timestamp'])
df1['day_of_week'] = df1['timestamp'].dt.weekday_name
#https://stackoverflow.com/questions/30222533/create-a-day-of-week-column-in-a-pandas-dataframe-using-python

In [62]:
df1['day_of_week'].value_counts()



Monday       357
Tuesday      326
Wednesday    322
Thursday     305
Friday       305
Saturday     284
Sunday       276
Name: day_of_week, dtype: int64

In [63]:
df1[df1['day_of_week']== 'Monday']['tweet_id'].count()

357

In [64]:
import plotly
plotly.tools.set_credentials_file(username='
import plotly.plotly as py
#from plotly.graph_objs import *
import plotly.graph_objs as go

data = [go.Bar(
            x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            y=[df1[df1['day_of_week']== 'Monday']['tweet_id'].count(), 
               df1[df1['day_of_week']== 'Tuesday']['tweet_id'].count(), 
               df1[df1['day_of_week']== 'Wednesday']['tweet_id'].count(), 
               df1[df1['day_of_week']== 'Thursday']['tweet_id'].count(), 
               df1[df1['day_of_week']== 'Friday']['tweet_id'].count(), 
               df1[df1['day_of_week']== 'Saturday']['tweet_id'].count(), 
               df1[df1['day_of_week']== 'Sunday']['tweet_id'].count()
              ]
    )]
layout = go.Layout(  
    title='Day of the Week by number of Tweet'
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Day of the Week by number of Tweet')


I wanted to see which day of the week people tweet the most and the trend is insightful. It starts with Monday as the highest count and gradually declines over the days!

Monday       384
Tuesday      356
Wednesday    354
Friday       333
Thursday     326
Saturday     309
Sunday       294

# Analysis 2)

What were the inaccurate results for all three models? 
How accurate is the predictor model? 


In [65]:
df2= df_img_algo_clean.copy()
df2.info()
#df2
df2[(df2['predict_1_breed'] == False) & (df2['predict_2_breed'] == False) & (df2['predict_3_breed'] == False)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id           2075 non-null object
jpg_url            2075 non-null object
img_num            2075 non-null int64
prediction_1       2075 non-null object
p1_confidence      2075 non-null float64
predict_1_breed    2075 non-null bool
prediction_2       2075 non-null object
p2_confidence      2075 non-null float64
predict_2_breed    2075 non-null bool
prediction_3       2075 non-null object
p3_confidence      2075 non-null float64
predict_3_breed    2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,prediction_1,p1_confidence,predict_1_breed,prediction_2,p2_confidence,predict_2_breed,prediction_3,p3_confidence,predict_3_breed
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,4.588540e-02,False,terrapin,1.788530e-02,False
17,666104133288665088,https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg,1,hen,0.965932,False,cock,3.391940e-02,False,partridge,5.206580e-05,False
18,666268910803644416,https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg,1,desktop_computer,0.086502,False,desk,8.554740e-02,False,bookcase,7.947970e-02,False
21,666293911632134144,https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg,1,three-toed_sloth,0.914671,False,otter,1.525000e-02,False,great_grey_owl,1.320720e-02,False
25,666362758909284353,https://pbs.twimg.com/media/CT9lXGsUcAAyUFt.jpg,1,guinea_pig,0.996496,False,skunk,2.402450e-03,False,hamster,4.608630e-04,False
29,666411507551481857,https://pbs.twimg.com/media/CT-RugiWIAELEaq.jpg,1,coho,0.404640,False,barracouta,2.714850e-01,False,gar,1.899450e-01,False
45,666786068205871104,https://pbs.twimg.com/media/CUDmZIkWcAAIPPe.jpg,1,snail,0.999888,False,slug,5.514170e-05,False,acorn,2.625800e-05,False
50,666837028449972224,https://pbs.twimg.com/media/CUEUva1WsAA2jPb.jpg,1,triceratops,0.442113,False,armadillo,1.140710e-01,False,common_iguana,4.325530e-02,False
51,666983947667116034,https://pbs.twimg.com/media/CUGaXDhW4AY9JUH.jpg,1,swab,0.589446,False,chain_saw,1.901420e-01,False,wig,3.450970e-02,False
53,667012601033924608,https://pbs.twimg.com/media/CUG0bC0U8AAw2su.jpg,1,hyena,0.987230,False,african_hunting_dog,1.260080e-02,False,coyote,5.735010e-05,False


There are 324 values that are false for all three predictions out of 2074. it's rougly about 15% completely incorrect. When the model is inaccruate, it predicts something other than a dog. 

In [66]:
df2['tweet_id'][(df2['predict_1_breed'] == True)].count()/2075
#model 1 is accurate .74 out of 1

0.7383132530120482

In [67]:
df2['tweet_id'][(df2['predict_2_breed'] == True)].count()/2075
#model 2 is accurate .75 out of 1

0.74843373493975907

In [68]:
df2['tweet_id'][(df2['predict_3_breed'] == True)].count()/2075
#model 3 is accurate .72 out of 1

0.72240963855421692

# Analysis 3)

How is WeRateDogs doing? have they been gaining followers? 

In [69]:
df3 = df_tweet_json_clean.copy()
#f3.columns
df3.sort_values(by = ['created_at'])

#Yes, they have been 


,tweet_id,retweet_count,favorite_count,followers_count,url,created_at
2344,666020888022790149,521,2560,6552609,https://t.co/BLDqew2Ijj,2015-11-15 22:32:08
2343,666029285002620928,47,131,6552609,https://t.co/r7mOb2m0UI,2015-11-15 23:05:30
2342,666033412701032449,45,125,6552609,https://t.co/y671yMhoiR,2015-11-15 23:21:54
2341,666044226329800704,141,302,6552609,https://t.co/DWnyCjf2mx,2015-11-16 00:04:52
2340,666049248165822465,40,109,6552609,https://t.co/4B7cOc1EDq,2015-11-16 00:24:50
2339,666050758794694657,58,133,6552609,https://t.co/fvIbQfHjIe,2015-11-16 00:30:50
2338,666051853826850816,859,1229,6552609,https://t.co/v5A4vzSDdc,2015-11-16 00:35:11
2337,666055525042405380,257,441,6552609,https://t.co/rdivxLiqEt,2015-11-16 00:49:46
2336,666057090499244032,142,300,6552609,https://t.co/yWBqbrzy8O,2015-11-16 00:55:59
2335,666058600524156928,58,114,6552608,https://t.co/pYAJkAe76p,2015-11-16 01:01:59


In [70]:
#df3.columns
import plotly.plotly as py
import plotly.graph_objs as go

df3.drop('tweet_id', axis = 1)
df3.drop('url', axis = 1)
df3.sort_values(by= ['created_at'])



,tweet_id,retweet_count,favorite_count,followers_count,url,created_at
2344,666020888022790149,521,2560,6552609,https://t.co/BLDqew2Ijj,2015-11-15 22:32:08
2343,666029285002620928,47,131,6552609,https://t.co/r7mOb2m0UI,2015-11-15 23:05:30
2342,666033412701032449,45,125,6552609,https://t.co/y671yMhoiR,2015-11-15 23:21:54
2341,666044226329800704,141,302,6552609,https://t.co/DWnyCjf2mx,2015-11-16 00:04:52
2340,666049248165822465,40,109,6552609,https://t.co/4B7cOc1EDq,2015-11-16 00:24:50
2339,666050758794694657,58,133,6552609,https://t.co/fvIbQfHjIe,2015-11-16 00:30:50
2338,666051853826850816,859,1229,6552609,https://t.co/v5A4vzSDdc,2015-11-16 00:35:11
2337,666055525042405380,257,441,6552609,https://t.co/rdivxLiqEt,2015-11-16 00:49:46
2336,666057090499244032,142,300,6552609,https://t.co/yWBqbrzy8O,2015-11-16 00:55:59
2335,666058600524156928,58,114,6552608,https://t.co/pYAJkAe76p,2015-11-16 01:01:59


In [71]:
data = [go.Scatter( x=df3['created_at'], y=df3['followers_count'] )]

layout = go.Layout(
    title='Day of the Week by number of Tweet'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Number of Followers by Year')
#this account has been losing followers


In [72]:
df_arch.pupper.value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [73]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2175 non-null object
timestamp             2175 non-null datetime64[ns]
source                2175 non-null category
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null float64
rating_denominator    2175 non-null float64
name                  2175 non-null object
rating_value          2175 non-null float64
dog_stage             364 non-null object
dtypes: category(1), datetime64[ns](1), float64(3), object(5)
memory usage: 172.2+ KB
